# Materials data science: descriptors and machine learning

Welcome to the materials data science lesson. In this session, we will demonstrate how to use `matminer`, `pandas` and `scikit-learn` for machine learning materials properties.

The lesson is split into three sections:
1. Data retrieval and basic analysis of pandas `DataFrame` objects.
2. Generating machine learnable descriptors.
3. Training, testing and visualizing machine learning methods with `scikit-learn` and `FigRecipes`.

Many more tutorials on how to use matminer (beyond the scope of this workshop) are available in the `matminer_examples` repository, available [here](https://github.com/hackingmaterials/matminer_examples).

## Machine learning workflow

Firstly, what does a typical machine learning workflow look like? The overall process can be summarized as:
1. Take raw inputs, such as a list of compositions, and an associated target property to learn.
2. Convert the raw inputs into *descriptors* or *features* that can be learned by machine learning algorithms.
3. Train a machine learning model on the data.
4. Plot and analyze the performance of the model.

<img src="resources/ml_workflow.png" alt="machine learning workflow" style="width: 700px;"/>


Typically, questions asked by a new practitioner in the field include:
- Where do we get the raw data from?
- How do we convert the raw data into learnable features?
- How can we plot and interpret the results of a model?

The `matminer` package has been developed to help make machine learning of materials properties easy and hassle free. The aim of matminer is to connect materials data with data mining algorithms and data visualization.

<img src="resources/matminer.png" alt="matminer overview" style="width: 700px;"/>

## Part 1: Data retrieval and filtering

Matminer interfaces with many materials databases,  including:
- Materials Project
- Citrine
- AFLOW
- Materials Data Facility (MDF)
- Materials Platform for Data Science (MPDS)

In addition, it also includes datasets from published literature. Matminer hosts a repository of 26 (and growing) datasets which comes from published and peer-reviewed machine learning investigations of materials properties or publications of high-throughput computing studies.

In this section, we will show how to access and manipulate the datasets from the published literature. More information on accessing other materials databases are detailed in the [matminer_examples](https://github.com/hackingmaterials/matminer_examples) repository.

A list of the literature-based datasets can be printed using the `get_available_datasets()` function. This also prints information about what the dataset contains, such as the number of samples, the target properties, and how the data was obtained (e.g., via theory or experiment).

In [ ]:
from matminer.datasets import get_available_datasets

get_available_datasets()

Datasets can be loaded using the `load_dataset()` function and the database name. To save installation space, the datasets are not automatically downloaded when matminer is installed. Instead, the first time the dataset is loaded, it will be downloaded from the internet and stored in the matminer installation directory.

Let's load the `dielectric_constant` dataset. It contains 1,056 structures with dielectric properties calculated with DFPT-PBE.

In [ ]:
from matminer.datasets import load_dataset

df = load_dataset("dielectric_constant")

For commonly used datasets, matminer provides convenience loader functions. For example, the dielectric dataset could also be loaded as:

In [ ]:
from matminer.datasets.convenience_loaders import load_dielectric_constant

df = load_dielectric_constant()

## Manipulating and examining pandas `DataFrame` objects

The datasets are made available as pandas `DataFrame` objects. You can think of these as a type of "spreadsheet" object in Python. DataFrames have several useful methods you can use to explore and clean the data, some of which we'll explore below.

### Inspecting the dataset

The `head()` function prints a summary of the first few rows of a data set. You can scroll across to see more columns. From this, it is easy to see the types of data available in in the dataset.

In [ ]:
df.head()

Sometimes, if a dataset is very large, you will be unable to see all the available columns. Instead, you can see the full list of columns using the `columns` attribute:

In [ ]:
df.columns

A pandas `DataFrame` includes a function called `describe()` that helps determine statistics for the various numerical/categorical columns in the data. Note that the `describe()` function only describes numerical columns by default.

Sometimes, the `describe()` function will reveal outliers that indicate mistakes in the data.

In [ ]:
df.describe()

### Indexing the dataset

We can access a particular column of `DataFrame` by indexing the object using the column name. For example:

In [ ]:
df["band_gap"]

Alternatively, we can access a particular row of a `Dataframe` using the `iloc` attribute.

In [ ]:
df.iloc[100]

### Filtering the dataset

Pandas `DataFrame` objects make it very easy to filter the data based on a specific column. We can use the typical Python comparison operators (==, >, >=, <, etc) to filter numerical values. For example, let's find all entries where the cell volume is greater than 580. We do this by filtering on the `volume` column.

Note that we first produce a *boolean mask* – a series of `True` and `False` depending on the comparison. We can then use the mask to filter the `DataFrame`. 

In [ ]:
mask = df["volume"] >= 580
df[mask]

We can use this method of filtering to clean our dataset. For example, if we only wanted our dataset to include semiconductors (materials with a non-zero band gap), we can do this easily by filtering the `band_gap` column.

In [ ]:
semiconductor_df = df[df["band_gap"] > 0]
semiconductor_df

Often, a dataset contains many additional columns that are not necessary for machine learning. Before we can train a model on the data, we need to remove any extraneous columns. We can remove whole columns from the dataset using the `drop()` function. This function can be used to drop both rows and columns.

The function takes a list of items to drop. For columns, this is column names whereas for rows it is the row number. Finally, the `axis` option specifies whether the data to drop is columns (`1`) or rows (`0`).

For example, to remove the `nsites`, `space_group`, `e_electronic`, and `e_total` columns, we can run: 

In [ ]:
cleaned_df = df.drop(["nsites", "space_group", "e_electronic", "e_total"],
                     axis=1)

Let's examine the cleaned `DataFrame` to see that the columns have been removed.

In [ ]:
cleaned_df.head()

### Generating new columns

Pandas `DataFrame` objects also make it easy to perform simple calculations on the data. Think of this as using formulas in Excel spreadsheets. All fundamental Python math operators (such as +, -, /, and \*) can be used. 

For example, the dielectric dataset contains the electronic contribution to the dielectric constant ($\epsilon_\mathrm{electronic}$, in the `poly_electronic` column) and the total (static) dielectric constant ($\epsilon_\mathrm{total}$, in the `poly_total` column). The ionic contribution to the dataset is given by:

$$
\epsilon_\mathrm{ionic} = \epsilon_\mathrm{total} - \epsilon_\mathrm{electronic}
$$

Below, we calculate the ionic contribution to the dielectric constant and store it in a new column called `poly_ionic`. This is as simple as assigning the data to the new column, even if the column doesn't already exist.

In [ ]:
df["poly_ionic"] = df["poly_total"] - df["poly_electronic"]

Let's check the new data was added correctly.

In [ ]:
df.head()

## Let's practice!

Now, let's practice. You'll download a dataset, inspect it, and make sure it is ready to be used for machine learning.